<a href="https://colab.research.google.com/github/ramo2004/House-Pricing-Prediction-Model-Comparison/blob/main/AI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#import all libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score



In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:


# Load the first dataset
#data1 = pd.read_csv('sample_data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
data1 = pd.read_csv('drive/MyDrive/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')


# Load the second dataset
#data2 = pd.read_csv('sample_data/realtor-data.zip.csv')
data2 = pd.read_csv('drive/MyDrive/realtor-data.zip.csv')



In [51]:
# Display the column names
print(data1.columns)

# Show the first few rows to get an idea of the data
print(data1.head())


Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '1/31/00',
       ...
       '12/31/23', '1/31/24', '2/29/24', '3/31/24', '4/30/24', '5/31/24',
       '6/30/24', '7/31/24', '8/31/24', '9/30/24'],
      dtype='object', length=306)
   RegionID  SizeRank  RegionName RegionType StateName State      City  \
0     91982         1       77494        zip        TX    TX      Katy   
1     61148         2        8701        zip        NJ    NJ  Lakewood   
2     91940         3       77449        zip        TX    TX      Katy   
3     62080         4       11368        zip        NY    NY  New York   
4     91733         5       77084        zip        TX    TX   Houston   

                                   Metro        CountyName      1/31/00  ...  \
0   Houston-The Woodlands-Sugar Land, TX  Fort Bend County  210115.6491  ...   
1  New York-Newark-Jersey City, NY-NJ-PA      Ocean County  138190.2243  ...   
2   Houston-Th

In [52]:
# Filter data for San Antonio in Texas
san_antonio_data=data1[(data1['City']=='San Antonio') & (data1['StateName']=='TX')]

# Display the filtered data
print(san_antonio_data.head())


     RegionID  SizeRank  RegionName RegionType StateName State         City  \
41      92341        42       78245        zip        TX    TX  San Antonio   
225     92350       231       78254        zip        TX    TX  San Antonio   
353     92345       361       78249        zip        TX    TX  San Antonio   
429     92349       439       78253        zip        TX    TX  San Antonio   
468     92347       478       78251        zip        TX    TX  San Antonio   

                             Metro    CountyName      1/31/00  ...  \
41   San Antonio-New Braunfels, TX  Bexar County  108657.7837  ...   
225  San Antonio-New Braunfels, TX  Bexar County  134158.0683  ...   
353  San Antonio-New Braunfels, TX  Bexar County  121232.4915  ...   
429  San Antonio-New Braunfels, TX  Bexar County  166330.0264  ...   
468  San Antonio-New Braunfels, TX  Bexar County  112480.2507  ...   

        12/31/23      1/31/24      2/29/24      3/31/24      4/30/24  \
41   267147.6152  266084.4442  2

In [53]:
# Step 1: Set the identifier columns as the index
san_antonio_data.set_index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'], inplace=True)

# Step 2: Use `stack` to turn the date columns into rows
san_antonio_prices = san_antonio_data.stack().reset_index()

# Step 3: Rename the columns for clarity
san_antonio_prices.columns = ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName', 'Date', 'Price']

# Step 4: Convert the `Date` column to datetime format
san_antonio_prices['Date'] = pd.to_datetime(san_antonio_prices['Date'], format='%m/%d/%y', errors='coerce')

# Step 5: Display the reshaped data
san_antonio_prices.head()


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Date,Price
0,92341,42,78245,zip,TX,TX,San Antonio,"San Antonio-New Braunfels, TX",Bexar County,2000-01-31,108657.7837
1,92341,42,78245,zip,TX,TX,San Antonio,"San Antonio-New Braunfels, TX",Bexar County,2000-02-29,108721.0889
2,92341,42,78245,zip,TX,TX,San Antonio,"San Antonio-New Braunfels, TX",Bexar County,2000-03-31,108728.6312
3,92341,42,78245,zip,TX,TX,San Antonio,"San Antonio-New Braunfels, TX",Bexar County,2000-04-30,108872.6398
4,92341,42,78245,zip,TX,TX,San Antonio,"San Antonio-New Braunfels, TX",Bexar County,2000-05-31,108391.4348


In [54]:
# Count the unique price values
unique_prices = san_antonio_prices['Price'].nunique()
print("Number of unique prices:", unique_prices)

# Count the unique dates and regions
unique_dates = san_antonio_prices['Date'].nunique()
unique_regions = san_antonio_prices['RegionID'].nunique()

print("Unique Dates:", unique_dates)
print("Unique Regions:", unique_regions)



Number of unique prices: 16476
Unique Dates: 297
Unique Regions: 56


Beginning processing for dataset #2


In [55]:
# Display the column names
print(data2.columns)

# Show the first few rows to get an idea of the data
print(data2.head())

Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')
   brokered_by    status     price  bed  bath  acre_lot     street  \
0     103378.0  for_sale  105000.0  3.0   2.0      0.12  1962661.0   
1      52707.0  for_sale   80000.0  4.0   2.0      0.08  1902874.0   
2     103379.0  for_sale   67000.0  2.0   1.0      0.15  1404990.0   
3      31239.0  for_sale  145000.0  4.0   2.0      0.10  1947675.0   
4      34632.0  for_sale   65000.0  6.0   2.0      0.05   331151.0   

         city        state  zip_code  house_size prev_sold_date  
0    Adjuntas  Puerto Rico     601.0       920.0            NaN  
1    Adjuntas  Puerto Rico     601.0      1527.0            NaN  
2  Juana Diaz  Puerto Rico     795.0       748.0            NaN  
3       Ponce  Puerto Rico     731.0      1800.0            NaN  
4    Mayaguez  Puerto Rico     680.0         NaN            NaN  


In [56]:
data2_cleaned = data2.drop_duplicates()

data2_cleaned['zip_code'] = pd.to_numeric(data2_cleaned['zip_code'], errors='coerce')
data2_cleaned = data2_cleaned.dropna(subset=['zip_code'])


In [57]:
data2_cleaned['zip_code'] = data2_cleaned['zip_code'].astype(int).astype(str)
print(data2_cleaned['zip_code'].unique()[:10])  # View first 10 unique ZIP codes


['601' '795' '731' '680' '612' '639' '730' '670' '662' '669']


In [58]:
# Check unique values in both columns
print("Unique RegionName:", san_antonio_prices['RegionName'].unique()[:5])
print("Unique zip_code:", data2_cleaned['zip_code'].unique()[:5])

# Check for common values
common_values = set(san_antonio_prices['RegionName']) & set(data2_cleaned['zip_code'])
print("Common values between RegionName and zip_code:", common_values)


Unique RegionName: [78245 78254 78249 78253 78251]
Unique zip_code: ['601' '795' '731' '680' '612']
Common values between RegionName and zip_code: set()


In [59]:
data2_cleaned['zip_code'] = data2_cleaned['zip_code'].str.split('.').str[0]
print(data2_cleaned['zip_code'].unique()[:10])



['601' '795' '731' '680' '612' '639' '730' '670' '662' '669']


In [60]:
san_antonio_prices['RegionName'] = san_antonio_prices['RegionName'].astype(str)
data2_cleaned['zip_code'] = data2_cleaned['zip_code'].astype(str)


In [61]:





merged_data = pd.merge(
    san_antonio_prices,
    data2_cleaned,
    how='inner',
    left_on='RegionName',
    right_on='zip_code'
)

print("Shape of merged_data:", merged_data.shape)  # Check the size of the merged data
print(merged_data.head())  # Preview the merged data

print(san_antonio_prices['RegionName'].dtypes)  # Check RegionName type
print(data2_cleaned['zip_code'].dtypes)        # Check zip_code type
print("First few RegionName values:", san_antonio_prices['RegionName'].unique()[:10])
print("First few zip_code values:", data2_cleaned['zip_code'].unique()[:10])


Shape of merged_data: (2136423, 23)
   RegionID  SizeRank RegionName RegionType StateName State         City  \
0     92341        42      78245        zip        TX    TX  San Antonio   
1     92341        42      78245        zip        TX    TX  San Antonio   
2     92341        42      78245        zip        TX    TX  San Antonio   
3     92341        42      78245        zip        TX    TX  San Antonio   
4     92341        42      78245        zip        TX    TX  San Antonio   

                           Metro    CountyName       Date  ...     price  bed  \
0  San Antonio-New Braunfels, TX  Bexar County 2000-01-31  ...  478500.0  3.0   
1  San Antonio-New Braunfels, TX  Bexar County 2000-01-31  ...  270000.0  4.0   
2  San Antonio-New Braunfels, TX  Bexar County 2000-01-31  ...  275000.0  3.0   
3  San Antonio-New Braunfels, TX  Bexar County 2000-01-31  ...  450000.0  4.0   
4  San Antonio-New Braunfels, TX  Bexar County 2000-01-31  ...  389900.0  4.0   

  bath  acre_lot    

In [62]:
# Columns to keep
columns_to_keep = ['zip_code', 'price', 'bed', 'bath', 'acre_lot', 'house_size', 'Date']

# Filter to keep only relevant columns
san_antonio_cleaned = merged_data[columns_to_keep]

# Drop rows with missing values in relevant columns
san_antonio_cleaned = san_antonio_cleaned.dropna(subset=['price', 'bed', 'bath', 'acre_lot', 'house_size'])

print(san_antonio_cleaned.head())


  zip_code     price  bed  bath  acre_lot  house_size       Date
0    78245  478500.0  3.0   3.0      0.14      2898.0 2000-01-31
1    78245  270000.0  4.0   3.0      0.14      2530.0 2000-01-31
2    78245  275000.0  3.0   2.0      0.11      1450.0 2000-01-31
3    78245  450000.0  4.0   3.0      0.16      3276.0 2000-01-31
4    78245  389900.0  4.0   3.0      0.19      2377.0 2000-01-31


In [63]:
# Define features and target
features = ['bed', 'bath', 'acre_lot', 'house_size']
target = 'price'
X = san_antonio_cleaned[features]
y = san_antonio_cleaned[target]


In [64]:
x = pd.get_dummies(san_antonio_cleaned[features+['zip_code']],drop_first=True)


In [65]:
x = san_antonio_cleaned[features + ['zip_code']]  # Check if `x` has data
print(x.shape)  # Displays the number of rows and columns in `x`


(1600533, 5)


In [66]:
scaler = StandardScaler()
scaled_x=scaler.fit_transform(x)

In [67]:
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.2, random_state=42)


In [68]:
 #linear regression model starts here
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_lr)




In [69]:

print("Linear Regression Results:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_lr))
print("R-squared:", r2_score(y_test, y_pred_lr))
print("Mean Absolute Error:", mae)


#why is our MSE so high?

#Could be because our model is too simple to catch all the complexities in the relationship
#(too few features? LR cant fit non linear relationship?)

# Could be beacause of outliers: disproportionately priced house (luxury homes can be very expensive for example)

# Is also because house prices are very high numbers, and MSE squares errors,
# so even small deviations between predicted and actual values can become very large


# why is R Squared 64%?

#this means that our model took into account 64% of variation in house prices.
# The remaining 36% cannot be explained by features in our model (for eg, crime rates, school districts, etc.)


#why is MAE 99040? This means that our house prices are off by about 99 thousand dollars.
# Model is likely not complex enough, or nuances of housing market are not captured fully in the model


#The metrics will now be compared between our different models and best metrics means best performing model


Linear Regression Results:
Mean Squared Error: 28172604961.3672
R-squared: 0.6435188689578817
Mean Absolute Error: 99040.40460419575


In [70]:
 #decision tree model starts here
model_dt = DecisionTreeRegressor(max_depth=10, random_state=42) #max depth of 10 to try and prevent overfitting
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_dt)



In [71]:
print("Decision Tree Results:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_dt))
print("R-squared:", r2_score(y_test, y_pred_dt))
print("Mean Absolute Error:", mae)


#MSE is a lot lower than Linear Regression. This is because DT can handle more complex relationships.
# However it is still relatively high, this is likely becaue of the scale of house prices or outliers

#R^2 is higher than LR as well. This shows that this model is a better fit for the data,
# and better explains the proportion of variance in house prices

#why is MAE 47891? This means that our house prices are off by about 48 thousand dollars.
# Model is likely not complex enough, or nuances of housing market are not captured fully in the model


Decision Tree Results:
Mean Squared Error: 4947009380.9260025
R-squared: 0.9374031793720615
Mean Absolute Error: 47891.750646170585


In [72]:
# random forest model begins here
model_rf = RandomForestRegressor(n_estimators=100, random_state=42) #using 100 trees, will take significant computing time
model_rf.fit(X_train,y_train)
y_pred_rf = model_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_rf)


In [73]:
print("Random Forest Results:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
print("R-squared:", r2_score(y_test, y_pred_rf))
print("Mean Absolute Error:", mae)

# very low MSE, this shows a much better result than decision trees or Linear Regression.
# very high r^2, this means the model explains 99.5% of variance in the house prices.
# low MAE, house prices are off by about only $1600.

# Why are the results so much better? because using multiple decision trees reduces proness to overfitting and smoothes out errors.
# However, the results could also be because of overfitting, the model should be tested on unseen data to confirm

Random Forest Results:
Mean Squared Error: 38105149.57001163
R-squared: 0.9995178377421657
Mean Absolute Error: 1654.3431910954437


How can we minimize overfitting (when the model memorizes the training data)? We can control hyperparameters. This will make the model more accurate. Some key hyperparameters include:     

min_samples_split,

max_depth,

max_features

We can use Grid or Randomized Search, these test combinations of parameters to see which provide the most optimal configuration.  

In [74]:
#some experimentation regarding hyperparameters, firstly the number of trees within RF (50 instead of 100).

model_rf = RandomForestRegressor(n_estimators=50, random_state=42)
model_rf.fit(X_train,y_train)
y_pred_rf = model_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_rf)


print("Experiment Random Forest Results:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
print("R-squared:", r2_score(y_test, y_pred_rf))
print("Mean Absolute Error:", mae)

Experiment Random Forest Results:
Mean Squared Error: 38108687.19637394
R-squared: 0.9995177929789267
Mean Absolute Error: 1654.4999022321313


The results from lowering the trees to 50 rather than 100. Around half the compute time (2 min 40 secs). Very similar results, the MAE and MSE were better in (n=100) by an insignificant amount. This is one example of tuning the model.

In [75]:
# Some experimentation with Decision Trees

 #decision tree model starts here
model_dt = DecisionTreeRegressor(max_depth=15, random_state=42) #max depth of 15 to see the difference that increasing max depth makes
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_dt)

print("Experiment Decision Tree Results:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_dt))
print("R-squared:", r2_score(y_test, y_pred_dt))
print("Mean Absolute Error:", mae)




Experiment Decision Tree Results:
Mean Squared Error: 1412441764.6512063
R-squared: 0.9821277145480709
Mean Absolute Error: 22311.417627848867


It seems that increasing the max depth (from 10 - 15) has increased the models performance.

After the above tests with DT and RF, we will check below to see if overfitting is occurring.


In [76]:
# Train DT with max depth of 10
model_dt = DecisionTreeRegressor(max_depth=10, random_state=42)
model_dt.fit(X_train, y_train)

# Evaluate on training set
y_train_pred = model_dt.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Evaluate on test set
y_test_pred = model_dt.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training MSE:", train_mse)
print("Training R²:", train_r2)
print("Test MSE:", test_mse)
print("Test R²:", test_r2)


Training MSE: 4942343033.184442
Training R²: 0.9373033876534966
Test MSE: 4947009380.9260025
Test R²: 0.9374031793720615


The above results show us that overfitting is not occuring in the DT model, because the scores for training and test are very similar.

The results are as follows (best performing model from top to bottom):

1) Random Forest

2) Decision Tree

3) Linear Regression


This does not come as a surpise as the housing market is complex and has many features etc, and simpler models cannot handle such relationships. The most complex model (in this case) is the best. However it does seem that with enough tuning, the DT could come relatively close to RF in terms of accuracy. Linear Regression lags behind both other models.

